In [3]:
from meta_opt.workloads.gnn import load_gnn
import jax

/Users/evandogariu/Desktop/meta-opt/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
cfg = {
    'num_iters': 500,
    'batch_size': 5,
    'num_eval_iters': -1,
    'full_batch': False
}

tr, te, e, l, m = load_gnn(cfg, dataset_dir='../datasets')

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


In [6]:
# Forked from the init2winit implementation here
# https://github.com/google/init2winit/blob/master/init2winit/model_lib/gnn.py.
from typing import Optional, Tuple

from flax import linen as nn
import jax.numpy as jnp
import jraph


def _make_embed(latent_dim, name):

  def make_fn(inputs):
    return nn.Dense(features=latent_dim, name=name)(inputs)

  return make_fn


def _make_mlp(hidden_dims, dropout, activation_fn):
  """Creates a MLP with specified dimensions."""

  def make_fn(*inputs):
    x = jnp.concatenate([inp.reshape(-1) for inp in inputs], axis=0).reshape(1, -1)
    for dim in hidden_dims:
      x = nn.Dense(features=dim)(x)
      x = nn.LayerNorm()(x)
      x = activation_fn(x)
      x = dropout(x)
    return x
      
  # @jraph.concatenated_args
  # def make_fn(inputs):
  #   x = inputs
  #   for dim in hidden_dims:
  #     x = nn.Dense(features=dim)(x)
  #     x = nn.LayerNorm()(x)
  #     x = activation_fn(x)
  #     x = dropout(x)
  #   return x

  return make_fn


class GNN(nn.Module):
  """Defines a graph network.
  The model assumes the input data is a jraph.GraphsTuple without global
  variables. The final prediction will be encoded in the globals.
  """
  num_outputs: int
  latent_dim: int = 256
  hidden_dims: Tuple[int] = (256,)
  # If None, defaults to 0.1.
  dropout_rate: Optional[float] = 0.1
  num_message_passing_steps: int = 5
  activation_fn_name: str = 'relu'

  @nn.compact
  def __call__(self, graph, train):
    if self.dropout_rate is None:
      dropout_rate = 0.1
    else:
      dropout_rate = self.dropout_rate
    dropout = nn.Dropout(rate=dropout_rate, deterministic=not train)

    graph = graph._replace(
        globals=jnp.zeros([graph.n_node.shape[1], self.num_outputs]))

    embedder = jraph.GraphMapFeatures(
        embed_node_fn=_make_embed(self.latent_dim, name='node_embedding'),
        embed_edge_fn=_make_embed(self.latent_dim, name='edge_embedding'))
    graph = embedder(graph)

    if self.activation_fn_name == 'relu':
      activation_fn = nn.relu
    elif self.activation_fn_name == 'gelu':
      activation_fn = nn.gelu
    elif self.activation_fn_name == 'silu':
      activation_fn = nn.silu
    else:
      raise ValueError(
          f'Invalid activation function name: {self.activation_fn_name}')

    for _ in range(self.num_message_passing_steps):
      net = jraph.GraphNetwork(
          update_edge_fn=_make_mlp(
              self.hidden_dims, dropout=dropout, activation_fn=activation_fn),
          update_node_fn=_make_mlp(
              self.hidden_dims, dropout=dropout, activation_fn=activation_fn),
          update_global_fn=_make_mlp(
              self.hidden_dims, dropout=dropout, activation_fn=activation_fn))

      graph = net(graph)

    # Map globals to represent the final result
    decoder = jraph.GraphMapFeatures(embed_global_fn=nn.Dense(self.num_outputs))
    graph = decoder(graph)

    return graph.globals

In [7]:
from meta_opt.nn import create_train_state
import optax
model = GNN(num_outputs=128)
opt = optax.adam(0.001)

In [ ]:
tstate = create_train_state(jax.random.PRNGKey(0), model, e, opt, l, metric_fns=m)

In [7]:
i = 0
for x in tr:
    print(x['weights'].shape, x['targets'].shape)
    i += 1
    if i > 5: break


(1, 6, 128) (1, 6, 128)
(1, 6, 128) (1, 6, 128)
(1, 6, 128) (1, 6, 128)
(1, 6, 128) (1, 6, 128)
(1, 6, 128) (1, 6, 128)
(1, 6, 128) (1, 6, 128)
